In [1]:
import json
import urllib.request

# Classe para simular a resposta da requisição.
class _Response:
    def __init__(self, data):
        self._data = data

    def json(self):
        return self._data

# Classe simulada para requests.
class requests:
    @staticmethod
    def get(url, *args, **kwargs):
        with urllib.request.urlopen(url) as resp:
            return _Response(json.load(resp))


dados = requests.get("https://servicodados.ibge.gov.br/api/v2/cnae/classes").json() # Resulta em uma lista de dicionários.

dados[0] # exibe o primeiro registro de dados(primeiro dicionário da lista).

{'id': '01113',
 'descricao': 'CULTIVO DE CEREAIS',
 'grupo': {'id': '011',
  'descricao': 'PRODUÇÃO DE LAVOURAS TEMPORÁRIAS',
  'divisao': {'id': '01',
   'descricao': 'AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS',
   'secao': {'id': 'A',
    'descricao': 'AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PESCA E AQÜICULTURA'}}},
 'observacoes': ['Esta classe compreende - o cultivo de alpiste, arroz, aveia, centeio, cevada, milho, milheto, painço, sorgo, trigo, trigo preto, triticale e outros cereais não especificados anteriormente',
  'Esta classe compreende ainda - o beneficiamento de cereais em estabelecimento agrícola, quando atividade complementar ao cultivo\r\n- a produção de sementes de cereais, quando atividade complementar ao cultivo',
  'Esta classe NÃO compreende - a produção de sementes certificadas dos cereais desta classe, inclusive modificadas geneticamente (01.41-5)\r\n- os serviços de preparação de terreno, cultivo e colheita realizados sob contrato (01.61-0)\r\n- o benefi

In [2]:
# Quantidade distinta de atividades, sabendo o tamanho da lista.
qtde_atividades_distintas = len(dados)

In [3]:
# Criação de uma lista dos grupos de atividades, extraindo a descrição de cada registro.

grupos = []

for registro in dados:
    grupos.append(registro["grupo"]["descricao"])

grupos[:10]

['PRODUÇÃO DE LAVOURAS TEMPORÁRIAS',
 'PRODUÇÃO DE LAVOURAS TEMPORÁRIAS',
 'PRODUÇÃO DE LAVOURAS TEMPORÁRIAS',
 'PRODUÇÃO DE LAVOURAS TEMPORÁRIAS',
 'PRODUÇÃO DE LAVOURAS TEMPORÁRIAS',
 'PRODUÇÃO DE LAVOURAS TEMPORÁRIAS',
 'PRODUÇÃO DE LAVOURAS TEMPORÁRIAS',
 'HORTICULTURA E FLORICULTURA',
 'HORTICULTURA E FLORICULTURA',
 'EXTRAÇÃO DE MINERAIS METÁLICOS NÃO-FERROSOS']

In [4]:
# A partir da lista, obtenha a quantidade de grupos de atividades.

qtde_grupos_distintos = len(set(grupos)) # o constructor set() cria uma estrutura de dados removendo as duplicatas.

In [5]:
# Resultado é uma lista de tuplas. Cria uma nova lista com o grupo e a quantidade de atividades em cada grupo.
grupos_count = [(grupo, grupos.count(grupo)) for grupo in set(grupos)]

grupos_count[:5]

[('FABRICAÇÃO DE PILHAS, BATERIAS E ACUMULADORES ELÉTRICOS', 2),
 ('ATIVIDADES DE ASSISTÊNCIA SOCIAL PRESTADAS EM RESIDÊNCIAS COLETIVAS E PARTICULARES',
  1),
 ('FABRICAÇÃO DE OUTROS PRODUTOS ALIMENTÍCIOS', 7),
 ('ATIVIDADES DE SERVIÇOS FINANCEIROS NÃO ESPECIFICADAS ANTERIORMENTE', 4),
 ('ATIVIDADES DE ORGANIZAÇÃO DE EVENTOS, EXCETO CULTURAIS E ESPORTIVOS', 1)]

In [6]:
# Transformamos a lista em um dicionario
grupos_count = dict(grupos_count)

In [7]:
# A partir do dicionário, podemos saber qual (ou quais) grupos possuem maior número de atividades.
valor_maximo = max(grupos_count.values())

grupos_mais_atividades = [
    chave for (chave, valor) in grupos_count.items() if valor == valor_maximo
]

print(len(grupos_mais_atividades))
print(grupos_mais_atividades)

1
['REPRESENTANTES COMERCIAIS E AGENTES DO COMÉRCIO, EXCETO DE VEÍCULOS AUTOMOTORES E MOTOCICLETAS']


In [8]:
from datetime import datetime

class ETL:
    def __init__(self):
        self.url = None

    def extract_cnae_data(self, url):
        self.url = url

        data_extracao = datetime.today().strftime("%Y/%m/%d - %H:%M:%S")

        # Extração dos dados.
        dados = requests.get(self.url).json()

        # Extrai os grupos de registros.
        grupos = []

        for registro in dados:
            grupos.append(registro["grupo"]["descricao"])

        # Cria uma lista de tuplas (grupo, quantidade de atividades).
        grupos_count = [(grupo, grupos.count(grupo)) for grupo in set(grupos)]
        
        grupos_count = dict(grupos_count) # Converte a lista de tuplas em dicionário.

        valor_maximo = max(grupos_count.values()) # Obtém o valor máximo de atividades.

        # Gera uma lista com os grupos que possuem o maior número de atividades.
        grupos_mais_atividades = [
            chave for (chave, valor) in grupos_count.items() if valor == valor_maximo
        ]


        # Informações finais e impressão dos resultados.
        qtde_atividades_distintas = len(dados)

        qtde_grupos_distintos = len(set(grupos))

        print(f"Dados extraídos em: {data_extracao}")
        print(f"Quantidade de atividades distintas = {qtde_atividades_distintas}")
        print(f"Quantidade de grupos distintos = {qtde_grupos_distintos}")
        print(
            f"Grupos com o maior número de atividades = {grupos_mais_atividades}, atividades = {valor_maximo}"
        )

        return None

# Usando a classe ETL para extrair os dados.
ETL().extract_cnae_data("https://servicodados.ibge.gov.br/api/v2/cnae/classes")

Dados extraídos em: 2025/11/07 - 15:00:30
Quantidade de atividades distintas = 673
Quantidade de grupos distintos = 285
Grupos com o maior número de atividades = ['REPRESENTANTES COMERCIAIS E AGENTES DO COMÉRCIO, EXCETO DE VEÍCULOS AUTOMOTORES E MOTOCICLETAS'], atividades = 9
